In [1]:
%matplotlib notebook
from mpl_toolkits import mplot3d


import numpy as np
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data 
import time
import math
import random


In [2]:
physicsClient = p.connect(p.GUI)

In [3]:
p.setGravity(0,0,-10) 
p.resetSimulation() 
p.setAdditionalSearchPath(pybullet_data.getDataPath()) 
planeId = p.loadURDF("plane.urdf") 
robotId = p.loadURDF("summer_intern/summer_project/kuka_iiwa/urdf/iiwa7.urdf",flags=9, useFixedBase=1)

robotStartPos = [0,0,0]
robotStartOrientation = p.getQuaternionFromEuler([0,0,0])

p.resetBasePositionAndOrientation(robotId,robotStartPos,robotStartOrientation)

p.setJointMotorControlArray(robotId,range(7),p.VELOCITY_CONTROL,forces=np.zeros(7))

error: Cannot load URDF file.

In [ ]:
def simulate_system(x, u):
    x_next=[]
    for i in range(7):
        p.resetJointState(robotId,i,x[i],targetVelocity = x[i+7])
    
    p.setJointMotorControlArray(robotId,range(7), controlMode=p.TORQUE_CONTROL,forces=u)
    p.stepSimulation()
    for i in range(7):
        x_next.append(p.getJointStates(robotId,range(7))[i][0])
    for i in range(7):
        x_next.append(p.getJointStates(robotId,range(7))[i][1])
    x_next = np.array(x_next)
    return x_next

In [ ]:
N=50
xi = np.zeros([14,N])
ui= np.zeros([7, N])
y=np.zeros([14,N])

In [ ]:
for i in range(N):
    for j in range(7):
        a=random.randint(-180,180)#angule range(-pi,pi)
        b=random.uniform(-1, 10)#velocity range
        c=random.randint(-200,200)#torque range (-200,200)
        xi[j,i]=math.radians(a/math.pi)
        xi[j+7,i]=b
        ui[j,i]=c

x=np.zeros([21,N])
for _ in range(N):
    x[:,_]=np.concatenate((xi[:,_], ui[:,_]))
print(np.shape(x))

In [ ]:
for i in range(N):
    y[:,i]=simulate_system(xi[:,i], ui[:,i])


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn,optim
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transforms
torch.manual_seed(1)

In [ ]:
x = x.astype(np.float32)
y=y.astype(np.float32)

In [ ]:
inputs=torch.from_numpy(x.T)
outputs=torch.from_numpy(y.T)
print(inputs.size())
print(outputs.size())
data_set=TensorDataset(inputs,outputs)

In [ ]:
class linear_regression(nn.Module):
    def __init__(self,input_size,output_size):
        super(linear_regression,self).__init__()
        self.linear=nn.Linear(input_size,output_size)
    def forward(self,x):
        yhat=self.linear(x)
        return yhat

In [ ]:
model=linear_regression(21,14)
optimizer = optim.SGD(model.parameters(), lr = 0.1)
criterion = nn.MSELoss()
train_loader=DataLoader(dataset=data_set,batch_size=15)


In [ ]:
LOSS=[]
 
epochs=50
   
for epoch in range(epochs):
    for x,y in train_loader:

        #make a prediction 
        yhat=model(x)
        #calculate the loss
        loss=criterion(yhat,y)
        #store loss/cost 
        LOSS.append(loss.item())
        #clear gradient 
        optimizer.zero_grad()
        #Backward pass: compute gradient of the loss with respect to all the learnable parameters
        loss.backward()
        #the step function on an Optimizer makes an update to its parameters
        optimizer.step()
     

In [ ]:
plt.plot(LOSS)
plt.xlabel("iterations ")
plt.ylabel("Cost/total loss ")
plt.show()

In [ ]:
p.disconnect()

In [ ]:
print(loss)